In [1]:
!pip install openai-whisper googletrans==4.0.0-rc1 spacy transformers tqdm langdetect google-generativeai chromadb gtts
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 37.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import whisper
import spacy
import google.generativeai as genai
from googletrans import Translator
from langdetect import detect
from tqdm import tqdm
import chromadb
from chromadb.utils import embedding_functions
from gtts import gTTS
import os


In [3]:

# Configure Gemini API
genai.configure(api_key="USe your api dont ask me or expect from me🫡🫡🫡")
model = genai.GenerativeModel('gemini-2.0-flash')




In [4]:
print("Loading models...")

# Whisper ASR for speech-to-text
whisper_model = whisper.load_model("large")

# SpaCy NLP Model
nlp = spacy.load("en_core_web_sm")

# Translator for multilingual support
translator = Translator()

# ChromaDB Client for storing conversations
client = chromadb.PersistentClient(path="./chroma_db")
collection = client.get_or_create_collection(name="real_estate_conversations")

print("Models loaded successfully!")


Loading models...


100%|█████████████████████████████████████| 2.88G/2.88G [00:59<00:00, 51.8MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Models loaded successfully!


In [5]:
def transcribe_audio(audio_path):
    """Transcribe multilingual audio using Whisper."""
    print("Transcribing audio...")
    result = whisper_model.transcribe(audio_path)
    return result['text']


In [6]:
def translate_text(text, dest_language='en'):
    """Detect language and translate text if needed."""
    detected_lang = detect(text)

    if detected_lang != dest_language:
        print(f"Detected Language: {detected_lang}. Translating to {dest_language}...")
        translated = translator.translate(text, dest=dest_language)
        return translated.text
    return text  # Return original text if already in the target language.


In [7]:
def extract_key_details(text):
    """Extracts key real estate details like location, budget, property type, and timeline."""
    print("Extracting key details...")
    doc = nlp(text)
    key_details = {
        "location": None,
        "property_type": None,
        "budget": None,
        "timeline": None
    }

    for ent in doc.ents:
        if ent.label_ == "GPE":  # Location
            key_details["location"] = ent.text
        elif ent.label_ == "MONEY":  # Budget
            key_details["budget"] = ent.text
        elif "BHK" in ent.text or "bedroom" in ent.text:  # Property type
            key_details["property_type"] = ent.text
        elif "month" in ent.text or "day" in ent.text or "week" in ent.text:
            key_details["timeline"] = ent.text

    return key_details


In [8]:
def generate_real_time_suggestion(context, user_language='en'):
    """Generates real estate suggestions and translates to the user's chosen language."""
    print("Generating real-time suggestions...")

    prompt = f"You are a real estate assistant. Based on this conversation, suggest what the agent should do:\n\n{context}"
    response = model.generate_content(prompt)

    suggestion = response.text

    if user_language != "en":
        suggestion = translate_text(suggestion, dest_language=user_language)

    return suggestion


In [16]:
def log_conversation(client_name, transcript, translated_text, key_details, follow_up):
    """Logs the conversation details in ChromaDB."""
    conversation_id = f"conv_{len(collection.get()['ids']) + 1}"

    # Convert key_details dictionary to string representation
    key_details_str = str(key_details)  # Convert to string

    collection.add(
        ids=[conversation_id],
        metadatas=[{
            "client_name": client_name,
            "transcript": transcript,
            "translated_text": translated_text,
            "key_details": key_details_str,  # Store as string
            "follow_up": follow_up
        }]
    )
    print("Conversation logged successfully.")

In [17]:
from google.colab import files

uploaded = files.upload()
audio_path = list(uploaded.keys())[0]
print(f"Uploaded file: {audio_path}")

transcript = transcribe_audio(audio_path)
print(f"Transcript: {transcript}")


Saving xyz.dat (online-audio-converter.com).mp3 to xyz.dat (online-audio-converter.com) (1).mp3
Uploaded file: xyz.dat (online-audio-converter.com) (1).mp3
Transcribing audio...
Transcript:  Hello, Thank you for reaching out to us. How can I assist you today? Namaste, मैं पूने में 2BHK फ्लाट ढूंढ रहा हूँ. मेरा बज़ेट 80 Lakhs तक है Got it. Are you looking for a ready to move in property or are you okay with under construction projects? Ready to move in चाहिए. मैं अगले 3 महीने में shift होना चाहता हूँ Understood. Do you have any specific areas in mind like कोत्रुड, हिंजवाडी और बानर? हाँ, कोत्रुड या बानर में देख सकते हैं पर सोसाइटी में अच्छा पार्क और सिक्यूरिटी होना चाहिए Sure, I'll note that down. Do you need any assistance with home loans or legal paperwork? Home loan की जरूरत नहीं है, legal paperwork के बारे में आप ही बता देना हूँ Alright. Alright, I'll send you some options by tomorrow. Can you share your email id or phone number for follow ups? हाँ, मेरा email id klient at gmail.com 

In [18]:
translated_text = translate_text(transcript, dest_language='en')
print(f"Translated Text: {translated_text}")

key_details = extract_key_details(translated_text)
print(f"Key Details: {key_details}")


Translated Text:  Hello, Thank you for reaching out to us. How can I assist you today? Namaste, मैं पूने में 2BHK फ्लाट ढूंढ रहा हूँ. मेरा बज़ेट 80 Lakhs तक है Got it. Are you looking for a ready to move in property or are you okay with under construction projects? Ready to move in चाहिए. मैं अगले 3 महीने में shift होना चाहता हूँ Understood. Do you have any specific areas in mind like कोत्रुड, हिंजवाडी और बानर? हाँ, कोत्रुड या बानर में देख सकते हैं पर सोसाइटी में अच्छा पार्क और सिक्यूरिटी होना चाहिए Sure, I'll note that down. Do you need any assistance with home loans or legal paperwork? Home loan की जरूरत नहीं है, legal paperwork के बारे में आप ही बता देना हूँ Alright. Alright, I'll send you some options by tomorrow. Can you share your email id or phone number for follow ups? हाँ, मेरा email id klient at gmail.com है और phone number 9876-543210 है Thank you. I'll get back to you soon with some great options. Have a nice day. धन्यवाद, आपका दिन भी अच्छा रहे
Extracting key details...
Key

In [19]:
user_language = input("Enter your preferred language for suggestions (e.g., en, hi, mr, te): ")
suggestion = generate_real_time_suggestion(translated_text, user_language)
print(f"Real-time Suggestion in {user_language}: {suggestion}")

# Convert to Speech & Play Suggestion
tts = gTTS(suggestion, lang=user_language)
tts.save("suggestion.mp3")
print("Suggestion saved as audio: suggestion.mp3")


Enter your preferred language for suggestions (e.g., en, hi, mr, te): en
Generating real-time suggestions...
Real-time Suggestion in en: Okay, based on the conversation, here's what the agent should do, broken down into actionable steps:

**Immediate Actions (Right Now):**

1.  **Save Contact Information:** Immediately save the client's email address (klient at gmail.com) and phone number (9876-543210) in the CRM or contact management system.  Double-check for accuracy.

**Tasks for Today (Priority):**

2.  **Property Search:**
    *   **Filter Criteria:** Conduct a thorough search for 2BHK ready-to-move-in flats in Kothrud and Baner, Pune.
    *   **Budget:** The price range should be up to 80 Lakhs.
    *   **Key Amenities:**  Prioritize properties in societies with good parks and security features.
    *   **Documentation:** When searching, begin gathering/saving information regarding legal paper work, if available.

**Tasks for Tomorrow (Before Contacting Client):**

3.  **Property